# 📘 Exercício Guiado: RAG de Regulamento de Fundos

## Performance – IA para Devs | Desenvolvimento e Orquestração de Agentes de IA para o Setor Financeiro

---

### 🎯 Objetivo do Exercício

Neste exercício guiado, você irá construir um sistema de **RAG (Retrieval-Augmented Generation)** para consultar e extrair informações estruturadas do **Regulamento do Fundo KNCR** (Kinea Rendimentos Imobiliários).

Ao final deste exercício, você será capaz de:

1. **Carregar e processar** documentos PDF com PyPDF
2. **Criar embeddings** e armazenar em um Vector Store
3. **Implementar RAG** para consultas sobre o regulamento
4. **Extrair dados estruturados** com Pydantic
5. **Desenvolver um assistente** especializado em regulamentos de fundos

---

### 🛠️ Stack Tecnológico

| Tecnologia | Uso |
|------------|-----|
| **LangChain** | Framework para RAG |
| **PyPDF** | Extração de texto de PDFs |
| **ChromaDB** | Vector Store para embeddings |
| **Pydantic** | Validação e estruturação de saídas |
| **OpenAI** | Embeddings e LLM |

---

### 📄 Sobre o Documento

O **KNCR11** (Kinea Rendimentos Imobiliários) é um Fundo de Investimento Imobiliário (FII) que investe predominantemente em Certificados de Recebíveis Imobiliários (CRI).

O regulamento contém informações importantes como:
- Política de investimentos
- Taxas de administração e gestão
- Regras de distribuição de rendimentos
- Direitos e deveres dos cotistas

---

## 🚀 Parte 0: Configuração do Ambiente

Primeiro, vamos instalar e importar as dependências necessárias.

In [ ]:
# Instalação das dependências
# Execute esta célula apenas uma vez

%pip install langchain langchain-openai langchain-community langchain-chroma --quiet
%pip install chromadb pypdf pydantic python-dotenv --quiet
%pip install tiktoken --quiet

In [ ]:
# Configuração das variáveis de ambiente

import os
from dotenv import load_dotenv

# Carrega variáveis do arquivo .env (se existir)
load_dotenv()

# Configure sua chave de API da OpenAI
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = "sua-chave-aqui"  # Substitua pela sua chave

print("✅ Ambiente configurado com sucesso!")

✅ Ambiente configurado com sucesso!


---

## 📚 Parte 1: Carregando o PDF do Regulamento

### 1.1 O que é PyPDF?

**PyPDF** é uma biblioteca Python para manipulação de arquivos PDF. No contexto de RAG, usamos para:

- 📖 **Extrair texto** de documentos PDF
- 📄 **Preservar metadados** como número da página
- 🔍 **Preparar conteúdo** para processamento pelo LLM

O LangChain oferece o `PyPDFLoader` que facilita a integração com o pipeline de RAG.

---

In [2]:
# Instalando pypdf diretamente no kernel atual
%pip install pypdf --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Carregando o PDF do Regulamento KNCR

from langchain_community.document_loaders import PyPDFLoader

# Caminho do arquivo PDF
CAMINHO_PDF = "KNCR_Regulamento_05-2025.pdf"

# Carrega o PDF
loader = PyPDFLoader(CAMINHO_PDF)
paginas = loader.load()

print("📄 REGULAMENTO KNCR CARREGADO")
print("=" * 60)
print(f"Total de páginas: {len(paginas)}")
print(f"\n📖 Preview da primeira página:")
print(f"{paginas[0].page_content[:500]}...")
print(f"\n📋 Metadados: {paginas[0].metadata}")

📄 REGULAMENTO KNCR CARREGADO
Total de páginas: 40

📖 Preview da primeira página:
1 
REGULAMENTO DO KINEA RENDIMENTOS IMOBILIÁRIOS FUNDO DE INVESTIMENTO IMOBILIÁRIO 
RESPONSABILIDADE LIMITADA 
CNPJ: 16.706.958/0001-32 
 
PARTE GERAL 
 
1. PRESTADORES DE SERVIÇOS ESSENCIAIS.  
 
1.1. ADMINISTRAÇÃO. A administração do KINEA RENDIMENTOS IMOBILIÁRIOS FUNDO DE 
INVESTIMENTO IMOBILIÁRIO  RESPONSABILIDADE LIMITADA  inscrito no Cadastro Nacional da Pessoa 
Jurídica do Ministério da Fazenda (“CNPJ”) sob o n° 16.706.958/0001-32 (“FUNDO”) será exercida pela INTRAG 
DISTRIBUIDORA DE TÍTU...

📋 Metadados: {'producer': 'Microsoft® Word para Microsoft 365', 'creator': 'Microsoft® Word para Microsoft 365', 'creationdate': '2025-05-14T09:27:21-03:00', 'msip_label_59f6b450-b779-4ed9-b37e-4a5b0cc9de23_enabled': 'true', 'msip_label_59f6b450-b779-4ed9-b37e-4a5b0cc9de23_setdate': '2023-09-21T16:31:37Z', 'msip_label_59f6b450-b779-4ed9-b37e-4a5b0cc9de23_method': 'Privileged', 'msip_label_59f6b450-b779-4ed9-b37

### 💡 Exercício 1.1: Explore o documento

Use a célula abaixo para explorar o conteúdo do documento. Tente encontrar:
- Em qual página está a política de investimentos?
- Onde estão definidas as taxas de administração?

In [ ]:
# 🎯 SEU CÓDIGO AQUI
# Explore o documento carregado
# Dica: use paginas[numero].page_content para ver o conteúdo de cada página

# Exemplo: visualizar página 5
# print(paginas[4].page_content)


---

## 📚 Parte 2: Chunking - Dividindo o Documento

### 2.1 Por que fazer Chunking?

Documentos grandes precisam ser divididos em **chunks** (pedaços menores) porque:

1. **Limite de contexto** - LLMs têm limite de tokens por requisição
2. **Precisão na busca** - Chunks menores permitem busca mais precisa
3. **Custo** - Enviar menos texto reduz custos de API

### 2.2 Parâmetros importantes:

| Parâmetro | Descrição | Valor típico |
|-----------|-----------|-------------|
| `chunk_size` | Tamanho máximo de cada chunk | 500-1000 caracteres |
| `chunk_overlap` | Sobreposição entre chunks | 50-100 caracteres |
| `separators` | Caracteres para dividir | `["\n\n", "\n", ". "]` |

A **sobreposição** garante que informações não sejam cortadas no meio de uma frase.

---

In [4]:
# Configurando o Text Splitter

from langchain_text_splitters import RecursiveCharacterTextSplitter

# Configuração do splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,           # Tamanho máximo de cada chunk
    chunk_overlap=100,         # Sobreposição entre chunks
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]  # Prioridade de separação
)

# Aplica o chunking
chunks = text_splitter.split_documents(paginas)

print("✂️ CHUNKING REALIZADO")
print("=" * 60)
print(f"Páginas originais: {len(paginas)}")
print(f"Chunks gerados: {len(chunks)}")
print(f"\n📄 Exemplo de chunk:")
print(f"Conteúdo: {chunks[20].page_content[:300]}...")
print(f"Metadados: {chunks[20].metadata}")

✂️ CHUNKING REALIZADO
Páginas originais: 40
Chunks gerados: 186

📄 Exemplo de chunk:
Conteúdo: item 1. 7.3., o FUNDO deve ser liquidado, nos termos da regulamentação aplicável , devendo o GESTOR 
permanecer no exercício de suas funções até a conclusão da liquidação  e o ADMINISTRADOR até o 
cancelamento do registro do fundo na CVM. 
 
1.7.5. No caso de descredenciamento de prestador de serviç...
Metadados: {'producer': 'Microsoft® Word para Microsoft 365', 'creator': 'Microsoft® Word para Microsoft 365', 'creationdate': '2025-05-14T09:27:21-03:00', 'msip_label_59f6b450-b779-4ed9-b37e-4a5b0cc9de23_enabled': 'true', 'msip_label_59f6b450-b779-4ed9-b37e-4a5b0cc9de23_setdate': '2023-09-21T16:31:37Z', 'msip_label_59f6b450-b779-4ed9-b37e-4a5b0cc9de23_method': 'Privileged', 'msip_label_59f6b450-b779-4ed9-b37e-4a5b0cc9de23_name': 'Compartilhamento Externo', 'msip_label_59f6b450-b779-4ed9-b37e-4a5b0cc9de23_siteid': '591669a0-183f-49a5-98f4-9aa0d0b63d81', 'msip_label_59f6b450-b779-4ed9-b37e-4a5b0

---

## 📚 Parte 3: Criando o Vector Store

### 3.1 O que são Embeddings?

**Embeddings** são representações numéricas (vetores) de texto que capturam o **significado semântico**.

```
"taxa de administração"  →  [0.23, -0.45, 0.12, ...] (1536 dimensões)
"custos de gestão"       →  [0.21, -0.42, 0.15, ...] (vetores similares!)
"previsão do tempo"      →  [-0.78, 0.33, -0.56, ...] (vetor diferente)
```

### 3.2 O que é um Vector Store?

Um **Vector Store** é um banco de dados especializado em armazenar e buscar vetores por similaridade.

O **ChromaDB** é uma opção leve e fácil de usar, ideal para desenvolvimento e prototipação.

---

In [ ]:
# Instalando dependências do Vector Store
%pip install langchain-chroma chromadb --quiet
%pip install --upgrade chromadb --quiet

In [5]:
# Criando o Vector Store com ChromaDB

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# Inicializa o modelo de embeddings
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Cria o vector store
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    collection_name="regulamento_kncr"
)

print("🗄️ VECTOR STORE CRIADO")
print("=" * 60)
print(f"Banco de dados: ChromaDB (em memória)")
print(f"Modelo de embedding: text-embedding-3-small")
print(f"Documentos indexados: {len(chunks)}")

🗄️ VECTOR STORE CRIADO
Banco de dados: ChromaDB (em memória)
Modelo de embedding: text-embedding-3-small
Documentos indexados: 186


### 💡 Exercício 3.1: Teste a busca semântica

Vamos testar se o Vector Store está funcionando corretamente.

In [7]:
# Testando a busca semântica

# Pergunta de teste
pergunta = "Qual é a taxa de administração do fundo?"

# Busca os documentos mais relevantes
docs_relevantes = vector_store.similarity_search(pergunta, k=3)

print("🔍 TESTE DE BUSCA SEMÂNTICA")
print("=" * 60)
print(f"\n❓ Pergunta: {pergunta}")
print(f"\n📄 Top 1 chunks mais relevantes:")

for i, doc in enumerate(docs_relevantes):
    print(f"\n--- Chunk {i+1} (Página {doc.metadata.get('page', 'N/A')}) ---")
    print(f"{doc.page_content[:400]}...")

🔍 TESTE DE BUSCA SEMÂNTICA

❓ Pergunta: Qual é a taxa de administração do fundo?

📄 Top 1 chunks mais relevantes:

--- Chunk 1 (Página 17) ---
aplicável.  
 
10.4. Tendo em vista que o FUNDO admite a aplicação nos Fundos Investidos que também cobram taxa de 
administração, taxa de gestão/ performance e/ou taxa de ingresso/saída, a Taxa Global prevista no item 10 
contemplará quaisquer taxas de administração  e gestão/performance e/ou taxa de ingresso/saída cobradas na 
realização de tais investimentos pelo FUNDO.  
 
10.5. O ADMINISTRADO...

--- Chunk 2 (Página 16) ---
as transferências não onerosas de Cotas por meio de doação, herança e sucessão. 
 
10. REMUNERAÇÃO 
 
10.1. Pela prestação dos serviços de administração fiduciária, escrituração e gestão, será devida pelo FUNDO 
uma taxa global correspondente a 1,00% (um por cento) ao ano sobre o valor contábil do patrimônio líquido do 
FUNDO, calculado conforme item 10.3 abaixo, ou calculada sobre o valor de merc...

--- Chunk 3 (Página

---

## 📚 Parte 4: Construindo a Chain RAG

### 4.1 Arquitetura da Chain RAG

```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│  Pergunta   │ ──▶ │  Retriever  │ ──▶ │   Prompt    │ ──▶ │     LLM     │
│  do Usuário │     │  (Busca)    │     │  (Contexto) │     │  (Resposta) │
└─────────────┘     └─────────────┘     └─────────────┘     └─────────────┘
```

### 4.2 Componentes:

1. **Retriever** - Busca documentos relevantes no Vector Store
2. **Prompt Template** - Formata a pergunta + contexto para o LLM
3. **LLM** - Gera a resposta baseada no contexto
4. **Output Parser** - Processa a saída do LLM

---

In [10]:
# Criando a Chain RAG

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# Configuração do retriever
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}  # Retorna os 5 documentos mais relevantes
)

# Prompt para RAG
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """Você é um especialista em fundos de investimento imobiliário (FII).
Use APENAS as informações fornecidas no contexto para responder.
Se a informação não estiver no contexto, diga que não encontrou no regulamento.
Sempre cite a página ou seção de onde tirou a informação quando possível.

Contexto do Regulamento:
{context}"""),
    ("human", "{question}")
])

# LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Função para formatar documentos
def format_docs(docs):
    formatted = []
    for doc in docs:
        page = doc.metadata.get('page', 'N/A')
        formatted.append(f"[Página {page}]\n{doc.page_content}")
    return "\n\n---\n\n".join(formatted)

# Chain RAG
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

print("✅ CHAIN RAG CONFIGURADA!")
print("   Retriever: similarity search (k=5)")
print("   LLM: gpt-4o-mini")

✅ CHAIN RAG CONFIGURADA!
   Retriever: similarity search (k=5)
   LLM: gpt-4o-mini


In [11]:
# Testando o RAG com perguntas sobre o regulamento

perguntas_teste = [
    "Qual é a taxa de administração do fundo KNCR?",
    "Quais tipos de ativos o fundo pode investir?",
    "Como são distribuídos os rendimentos aos cotistas?",
    "Qual é o prazo de duração do fundo?",
]

print("🤖 SISTEMA RAG - CONSULTA AO REGULAMENTO KNCR")
print("=" * 60)

for pergunta in perguntas_teste:
    print(f"\n❓ {pergunta}")
    resposta = rag_chain.invoke(pergunta)
    print(f"\n💬 {resposta}")
    print("\n" + "-" * 60)

🤖 SISTEMA RAG - CONSULTA AO REGULAMENTO KNCR

❓ Qual é a taxa de administração do fundo KNCR?

💬 A taxa de administração do fundo Kinea Rendimentos Imobiliários (KNCR) é de 1,00% (um por cento) ao ano sobre o valor contábil do patrimônio líquido do fundo, conforme descrito na seção 10.1 do regulamento (página 16).

------------------------------------------------------------

❓ Quais tipos de ativos o fundo pode investir?

💬 O fundo pode investir nos seguintes tipos de ativos:

1. CRI Elegíveis (Certificados de Recebíveis Imobiliários);
2. LCI Elegíveis (Letras de Crédito Imobiliário);
3. LH Elegíveis (Letras Hipotecárias);
4. Outros ativos financeiros, títulos e valores mobiliários permitidos pela regulamentação aplicável, que tenham rendimento pré-determinado ou rentabilidade alvo pré-determinada, e que possuam classificação de risco "AA" emitida pela Standard&Poors, Fitch ou equivalente pela Moody's (conforme página 10, item 6.2). 

Além disso, o fundo pode eventualmente compor sua 

### 💡 Exercício 4.1: Faça suas próprias perguntas

Use o sistema RAG para fazer perguntas sobre o regulamento.

In [16]:
# 🎯 SEU CÓDIGO AQUI
# Faça suas próprias perguntas ao sistema RAG

minha_pergunta = ""  # Preencha aqui

if minha_pergunta:
    resposta = rag_chain.invoke(minha_pergunta)
    print(f"❓ {minha_pergunta}")
    print(f"\n💬 {resposta}")

❓ quem é o administrador do fundo?

💬 O administrador do fundo é a KINEA INVESTIMENTOS LTDA., conforme mencionado na página 0 do regulamento.


---

## 📚 Parte 5: Extração Estruturada com Pydantic

### 5.1 Por que usar Pydantic?

Em sistemas de produção, precisamos de **dados estruturados** para:
- Integrar com outros sistemas
- Armazenar em bancos de dados
- Gerar relatórios automatizados

**Pydantic** permite definir **schemas** que forçam o LLM a retornar dados no formato esperado.

### 5.2 Estrutura do Schema:

```python
class MeuSchema(BaseModel):
    campo_obrigatorio: str = Field(description="Descrição para o LLM")
    campo_opcional: Optional[float] = Field(default=None, description="...")
    campo_enum: TipoEnum = Field(description="Valor restrito")
```

---

In [32]:
# Definindo o schema para extração de informações do fundo

from pydantic import BaseModel, Field
from typing import Optional, List
from enum import Enum

class TipoFundo(str, Enum):
    FII = "FII"
    FIP = "FIP"
    FIDC = "FIDC"
    FIAGRO = "FIAGRO"
    OUTRO = "OUTRO"

class PerfilRisco(str, Enum):
    CONSERVADOR = "CONSERVADOR"
    MODERADO = "MODERADO"
    ARROJADO = "ARROJADO"

class AtivoPermitido(BaseModel):
    """Ativo permitido para investimento pelo fundo."""
    tipo: str = Field(description="Tipo do ativo (CRI, LCI, Imóveis, etc.)")
    limite_percentual: Optional[float] = Field(default=None, description="Limite percentual do patrimônio")

class TaxaFundo(BaseModel):
    """Taxa cobrada pelo fundo."""
    nome: str = Field(description="Nome da taxa (administração, gestão, performance, etc.)")
    valor_percentual: float = Field(description="Valor percentual da taxa ao ano")
    base_calculo: Optional[str] = Field(default=None, description="Base de cálculo da taxa")

class InformacoesFundo(BaseModel):
    """Schema completo para extração de informações de regulamento de fundo."""
    
    # Identificação
    nome_fundo: str = Field(description="Nome completo do fundo")
    cnpj: Optional[str] = Field(default=None, description="CNPJ do fundo")
    codigo_negociacao: Optional[str] = Field(default=None, description="Código de negociação em bolsa (ticker)")
    tipo_fundo: TipoFundo = Field(description="Tipo do fundo de investimento")
    
    # Administração
    administrador: str = Field(description="Nome do administrador do fundo")
    gestor: Optional[str] = Field(default=None, description="Nome da empresa responsável pela GESTÃO da carteira do fundo. Procure no texto por 'GESTÃO' ou 'gestão da carteira do FUNDO será exercida pela' seguido do nome da empresa gestora (ex: KINEA INVESTIMENTOS LTDA)")
    custodiante: Optional[str] = Field(default=None, description="Nome do custodiante")
    
    # Taxas
    taxas: List[TaxaFundo] = Field(description="Lista de taxas cobradas pelo fundo")
    
    # Política de Investimentos
    objetivo: str = Field(description="Objetivo do fundo")
    ativos_permitidos: List[AtivoPermitido] = Field(description="Lista de ativos permitidos para investimento")
    perfil_risco: PerfilRisco = Field(description="Perfil de risco do fundo")
    
    # Distribuição
    periodicidade_distribuicao: Optional[str] = Field(default=None, description="Periodicidade de distribuição de rendimentos")
    percentual_minimo_distribuicao: Optional[float] = Field(default=None, description="Percentual mínimo de distribuição do resultado")
    
    # Prazo e Liquidez
    prazo_duracao: Optional[str] = Field(default=None, description="Prazo de duração do fundo")
    
    # Observações
    observacoes: Optional[str] = Field(default=None, description="Observações importantes sobre o fundo")

print("✅ Schema InformacoesFundo definido!")
print(f"\n📋 Total de campos: {len(InformacoesFundo.model_fields)}")

✅ Schema InformacoesFundo definido!

📋 Total de campos: 15


### 5.3 Usando `with_structured_output()`

O método `with_structured_output()` do LangChain faz o LLM retornar **diretamente um objeto Pydantic**, garantindo:

- ✅ Formato JSON válido
- ✅ Tipos corretos (string, float, int, etc.)
- ✅ Campos obrigatórios preenchidos
- ✅ Valores de Enum respeitados

In [33]:
# Criando o extrator estruturado

from langchain_openai import ChatOpenAI

# LLM configurado para saída estruturada
llm_estruturado = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Cria uma versão do LLM que retorna objetos Pydantic
extrator = llm_estruturado.with_structured_output(InformacoesFundo)

# Prompt para extração
prompt_extracao = ChatPromptTemplate.from_messages([
    ("system", """Você é um especialista em análise de regulamentos de fundos de investimento.
Extraia TODAS as informações solicitadas do regulamento fornecido.
Seja preciso com valores numéricos e percentuais.

Regulamento:
{contexto}"""),
    ("human", "Extraia as informações principais deste regulamento de fundo.")
])

print("🔧 Extrator estruturado configurado!")

🔧 Extrator estruturado configurado!


In [37]:
# Executando a extração estruturada

# Busca documentos relevantes para extração - incluindo termos sobre gestão
docs_para_extracao = vector_store.similarity_search(
    "taxa administração gestão objetivo política investimento rendimentos distribuição nome fundo CNPJ administrador gestor custodiante",
    k=20  # Mais documentos para ter contexto completo
)

# Busca adicional específica para o gestor
docs_gestor = vector_store.similarity_search(
    "gestão da carteira do fundo será exercida KINEA gestor",
    k=5
)

# Combina os documentos (removendo duplicatas pelo conteúdo)
conteudos_vistos = set()
docs_combinados = []
for doc in docs_para_extracao + docs_gestor:
    if doc.page_content not in conteudos_vistos:
        conteudos_vistos.add(doc.page_content)
        docs_combinados.append(doc)

# Formata o contexto
contexto_extracao = format_docs(docs_combinados)

# Cria a chain de extração
chain_extracao = prompt_extracao | extrator

# Executa a extração
info_fundo = chain_extracao.invoke({"contexto": contexto_extracao})

print("✅ EXTRAÇÃO ESTRUTURADA CONCLUÍDA")
print("=" * 60)
print(f"   Documentos utilizados: {len(docs_combinados)}")

✅ EXTRAÇÃO ESTRUTURADA CONCLUÍDA
   Documentos utilizados: 23


In [38]:
# Exibindo os dados extraídos

print("📊 INFORMAÇÕES EXTRAÍDAS DO FUNDO KNCR")
print("=" * 60)

print(f"\n🏦 IDENTIFICAÇÃO")
print(f"   Nome: {info_fundo.nome_fundo}")
print(f"   CNPJ: {info_fundo.cnpj}")
print(f"   Ticker: {info_fundo.codigo_negociacao}")
print(f"   Tipo: {info_fundo.tipo_fundo.value}")

print(f"\n👔 ADMINISTRAÇÃO")
print(f"   Administrador: {info_fundo.administrador}")
print(f"   Gestor: {info_fundo.gestor}")
print(f"   Custodiante: {info_fundo.custodiante}")

print(f"\n💰 TAXAS")
for taxa in info_fundo.taxas:
    base = f" ({taxa.base_calculo})" if taxa.base_calculo else ""
    print(f"   • {taxa.nome}: {taxa.valor_percentual}% a.a.{base}")

print(f"\n🎯 OBJETIVO")
print(f"   {info_fundo.objetivo}")

print(f"\n📈 ATIVOS PERMITIDOS")
for ativo in info_fundo.ativos_permitidos:
    limite = f" (limite: {ativo.limite_percentual}%)" if ativo.limite_percentual else ""
    print(f"   • {ativo.tipo}{limite}")

print(f"\n💵 DISTRIBUIÇÃO")
print(f"   Periodicidade: {info_fundo.periodicidade_distribuicao}")
print(f"   Mínimo: {info_fundo.percentual_minimo_distribuicao}%" if info_fundo.percentual_minimo_distribuicao else "   Mínimo: Não especificado")

print(f"\n⏱️ PRAZO")
print(f"   Duração: {info_fundo.prazo_duracao}")

print(f"\n⚠️ PERFIL DE RISCO: {info_fundo.perfil_risco.value}")

📊 INFORMAÇÕES EXTRAÍDAS DO FUNDO KNCR

🏦 IDENTIFICAÇÃO
   Nome: KINEA RENDIMENTOS IMOBILIÁRIOS FUNDO DE INVESTIMENTO IMOBILIÁRIO RESPONSABILIDADE LIMITADA
   CNPJ: 16.706.958/0001-32
   Ticker: None
   Tipo: FII

👔 ADMINISTRAÇÃO
   Administrador: INTRAG DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS LTDA.
   Gestor: KINEA INVESTIMENTOS LTDA.
   Custodiante: None

💰 TAXAS
   • Taxa Global: 1.0% a.a. (valor contábil do patrimônio líquido do FUNDO)

🎯 OBJETIVO
   Investir em ativos imobiliários e ativos de liquidez.

📈 ATIVOS PERMITIDOS
   • Imóveis
   • Ativos de Liquidez

💵 DISTRIBUIÇÃO
   Periodicidade: None
   Mínimo: Não especificado

⏱️ PRAZO
   Duração: Indeterminado

⚠️ PERFIL DE RISCO: MODERADO


In [39]:
# Exportando como JSON

print("📤 EXPORTANDO COMO JSON")
print("=" * 60)

json_resultado = info_fundo.model_dump_json(indent=2)
print(json_resultado)

# Salvando em arquivo
with open("kncr_info_extraida.json", "w", encoding="utf-8") as f:
    f.write(json_resultado)

print("\n✅ Arquivo salvo: kncr_info_extraida.json")

📤 EXPORTANDO COMO JSON
{
  "nome_fundo": "KINEA RENDIMENTOS IMOBILIÁRIOS FUNDO DE INVESTIMENTO IMOBILIÁRIO RESPONSABILIDADE LIMITADA",
  "cnpj": "16.706.958/0001-32",
  "codigo_negociacao": null,
  "tipo_fundo": "FII",
  "administrador": "INTRAG DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS LTDA.",
  "gestor": "KINEA INVESTIMENTOS LTDA.",
  "custodiante": null,
  "taxas": [
    {
      "nome": "Taxa Global",
      "valor_percentual": 1.0,
      "base_calculo": "valor contábil do patrimônio líquido do FUNDO"
    }
  ],
  "objetivo": "Investir em ativos imobiliários e ativos de liquidez.",
  "ativos_permitidos": [
    {
      "tipo": "Imóveis",
      "limite_percentual": null
    },
    {
      "tipo": "Ativos de Liquidez",
      "limite_percentual": null
    }
  ],
  "perfil_risco": "MODERADO",
  "periodicidade_distribuicao": null,
  "percentual_minimo_distribuicao": null,
  "prazo_duracao": "Indeterminado",
  "observacoes": "O fundo não permite resgates de cotas, exceto em caso de liq

---

## 📚 Parte 6: Assistente Especializado em Regulamentos

Vamos criar um assistente completo que combina RAG + Extração Estruturada para análise de regulamentos de fundos.

In [40]:
# Classe do Assistente de Regulamentos

class AssistenteRegulamentos:
    """
    Assistente especializado em análise de regulamentos de fundos.
    
    Funcionalidades:
    - Consultas via RAG
    - Extração estruturada de informações
    - Comparação de fundos
    """
    
    def __init__(self, vector_store):
        self.vector_store = vector_store
        self.retriever = vector_store.as_retriever(search_kwargs={"k": 5})
        self.llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        self._setup_chains()
    
    def _setup_chains(self):
        """Configura as chains de consulta e extração."""
        
        # Chain de consulta RAG
        prompt_consulta = ChatPromptTemplate.from_messages([
            ("system", """Você é um especialista em fundos de investimento.
Responda baseado APENAS no regulamento fornecido.
Seja preciso e cite as seções/páginas quando possível.

Regulamento:
{context}"""),
            ("human", "{question}")
        ])
        
        self.chain_consulta = (
            {"context": self.retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_consulta
            | self.llm
            | StrOutputParser()
        )
        
        # Chain de extração estruturada
        self.extrator = self.llm.with_structured_output(InformacoesFundo)
    
    def consultar(self, pergunta: str) -> str:
        """Faz uma consulta ao regulamento via RAG."""
        return self.chain_consulta.invoke(pergunta)
    
    def extrair_info(self) -> InformacoesFundo:
        """Extrai informações estruturadas do regulamento."""
        docs = self.vector_store.similarity_search(
            "taxa administração gestão objetivo política investimento",
            k=15
        )
        contexto = format_docs(docs)
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", "Extraia as informações do regulamento:\n\n{contexto}"),
            ("human", "Extraia todas as informações principais.")
        ])
        
        chain = prompt | self.extrator
        return chain.invoke({"contexto": contexto})
    
    def gerar_resumo(self) -> str:
        """Gera um resumo executivo do fundo."""
        info = self.extrair_info()
        
        resumo = []
        resumo.append("=" * 60)
        resumo.append(f"RESUMO EXECUTIVO - {info.nome_fundo}")
        resumo.append("=" * 60)
        resumo.append(f"\n🏦 Tipo: {info.tipo_fundo.value}")
        resumo.append(f"👔 Gestor: {info.gestor}")
        resumo.append(f"⚠️ Risco: {info.perfil_risco.value}")
        
        if info.taxas:
            total_taxas = sum(t.valor_percentual for t in info.taxas)
            resumo.append(f"💰 Custo Total (taxas): {total_taxas:.2f}% a.a.")
        
        resumo.append(f"\n🎯 Objetivo: {info.objetivo[:200]}...")
        
        return "\n".join(resumo)

print("✅ Classe AssistenteRegulamentos definida!")

✅ Classe AssistenteRegulamentos definida!


In [41]:
# Instanciando o assistente

assistente = AssistenteRegulamentos(vector_store=vector_store)

print("🤖 ASSISTENTE DE REGULAMENTOS")
print("=" * 60)
print("Sistema pronto para consultas!")

🤖 ASSISTENTE DE REGULAMENTOS
Sistema pronto para consultas!


In [42]:
# Gerando resumo executivo

print("\n📋 GERANDO RESUMO EXECUTIVO...")
print(assistente.gerar_resumo())


📋 GERANDO RESUMO EXECUTIVO...
RESUMO EXECUTIVO - Fundo de Investimento Imobiliário

🏦 Tipo: FII
👔 Gestor: Kinea Investimentos Ltda
⚠️ Risco: MODERADO
💰 Custo Total (taxas): 3.00% a.a.

🎯 Objetivo: Investir em ativos imobiliários e ativos de liquidez...


In [31]:
# Interface de consulta interativa

print("\n💬 MODO DE CONSULTA INTERATIVO")
print("=" * 60)
print("Exemplos de perguntas:")
print("  - Quais são as taxas do fundo?")
print("  - O fundo pode investir em ações?")
print("  - Como funcionam as assembleias de cotistas?")
print("  - Quais são os riscos do fundo?")

# Faça sua consulta
pergunta = "Quais são as taxas cobradas pelo fundo?"  # Modifique aqui

print(f"\n❓ {pergunta}")
print(f"\n💬 {assistente.consultar(pergunta)}")


💬 MODO DE CONSULTA INTERATIVO
Exemplos de perguntas:
  - Quais são as taxas do fundo?
  - O fundo pode investir em ações?
  - Como funcionam as assembleias de cotistas?
  - Quais são os riscos do fundo?

❓ Quais são as taxas cobradas pelo fundo?

💬 O fundo cobra as seguintes taxas:

1. **Taxa Global**: Correspondente a 1,00% (um por cento) ao ano sobre o valor contábil do patrimônio líquido do fundo, conforme descrito na seção 10.1 (página 16).

2. **Taxa de Custódia**: O fundo pagará ao custodiante uma taxa máxima de até 0,08% (oito centésimos por cento) ao ano do patrimônio líquido do fundo, com um mínimo mensal de até R$ 20.000,00 (vinte mil reais), conforme descrito na seção 10.7 (página 17).

Além disso, a Taxa Global pode incluir taxas de administração e gestão/performance cobradas pelos Fundos Investidos em que o fundo aplica, conforme mencionado na seção 10.4 (página 17).


### 💡 Exercício Final: Crie suas próprias consultas

Use o assistente para explorar o regulamento do fundo KNCR.

In [43]:
# 🎯 SEU CÓDIGO AQUI
# Faça suas próprias consultas ao assistente

minhas_perguntas = [
    # Adicione suas perguntas aqui
    "",
    "",
]

for p in minhas_perguntas:
    if p:
        print(f"\n❓ {p}")
        print(f"\n💬 {assistente.consultar(p)}")
        print("-" * 60)

---

## 📝 Resumo do Exercício

### O que você aprendeu:

1. **Carregamento de PDFs**
   - Usar `PyPDFLoader` para extrair texto de documentos PDF
   - Preservar metadados como número da página

2. **Chunking**
   - Dividir documentos grandes em pedaços menores
   - Configurar `chunk_size` e `chunk_overlap`

3. **Vector Store**
   - Criar embeddings com OpenAI
   - Armazenar e buscar com ChromaDB

4. **RAG**
   - Combinar busca semântica com geração de texto
   - Criar chains de consulta contextual

5. **Extração Estruturada**
   - Definir schemas com Pydantic
   - Usar `with_structured_output()` para garantir formato

---

### 🎯 Próximos Passos

- Adicionar mais regulamentos de fundos ao Vector Store
- Implementar comparação entre fundos
- Criar alertas automáticos para mudanças em regulamentos
- Integrar com APIs de dados de mercado

---

**Parabéns por completar o exercício!** 🎉